<a href="https://colab.research.google.com/github/bogard75/bogard75.github.io/blob/master/naver_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Import

In [0]:
import numpy as np
import pandas as pd
from datetime import datetime

import requests
import threading
import numpy as np
import io
from bs4 import BeautifulSoup


# 2 네이버 주식자료 입수 함수
## naver_sise_market_sum() : 종목정보 가져오기
'종목코드', '종목명', '현재가', '전일비', '등락률', '액면가', '시가총액', '상장주식수', '외국인비율', '거래량', 'PER', 'ROE', '토론방'

## naver_sise_time() : 시간대별 가격 가져오기


In [0]:

# 종목코드
def naver_sise_market_sum():
    d = pd.DataFrame() 
       
    for c in range(1, 50):
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=%s" % (c)
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text.encode('utf-8'), 'html.parser')
        tbl = soup.select('.type_2')[0]
        tbl_rows = tbl.find_all('tr')
        l = []
        for tr in tbl_rows:
            td = tr.find_all('td')
            row = [tr.text.replace('\n','').replace('\t','') for tr in td]
            try:
                stock_code = tr.select('.tltle')[0]['href'].split('=')[1]
                row.insert(1, stock_code)
                l.append(row)
            except:
                pass
            
        try:
            columns = ['N','종목코드','종목명','현재가','전일비','등락률','액면가','시가총액','상장주식수','외국인비율','거래량','PER','ROE','토론방']
            df = pd.DataFrame(l, columns=columns)
            df = df[pd.to_numeric(df.N) > 0].set_index('N').drop(columns='토론방')
            d = d.append(df)
        except:
            pass
        
    return d

def to_df(f):
    def wrapper(*args, **kwargs):
        _r = f(*args, **kwargs)
        # nparray를 데이터프레임으로, 이를 to_pickle()
        columns = ['기준일자','종목코드','체결시각','체결가','전일비','매도','매수','거래량','변동량']
        _df = pd.DataFrame(_r, columns=columns)
        _df = _df[_df.기준일자.str.len()>=8].reset_index(drop=True)
        to_numeric = ['체결가','전일비','매도','매수','거래량','변동량']
        for c in to_numeric:
            _df[c] = _df[c].str.replace(',','').apply(pd.to_numeric)
        return _df
    
    return wrapper


@to_df
def naver_sise_time(stock, date):
    #print("[%s] %s" % (stock, date))
    
    data = []
    for c in range(1, 50):
        url = "http://finance.naver.com/item/sise_time.nhn?code=%s&thistime=%s160000&page=%s" \
              % (stock, date, c)
        response = requests.get(url)
        soup = BeautifulSoup(response.text.encode('utf-8'), 'html.parser')
        quotes = soup.find_all('span', {'class': ['tah', 'p10']})

        for li in quotes:
            data.append(li.text.strip())
        try:
            if data[-7] == '09:00':
                break
        except:
            pass
        
    # 체결시각 체결가 전일비 매도 매수 거래량 변동량
    if len(data) == 0 :
        r = np.zeros(shape=(1,9))
    else:
        quote = np.reshape(np.array(data), (-1, 7)).tolist()
        r = [[date] + [stock] + i for i in quote]
        r = np.array(r)
    #print(r.shape)
    
    return r


def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()


In [0]:

if __name__=='__main__':
    #crdt = '20200519'
    crdt = time.strftime('%Y%m%d')
    
    # 네이버 코스피 종목코드를 받아서 to_pickle()
    stocks = naver_sise_market_sum()
    stocks.to_pickle("./stocks_%s" % (crdt))

    # 기간동안 거래량을 입수하여 nparray로 
    #days = pd.date_range('20200210', '20200210', ).strftime('%Y%m%d')
    #r = np.vstack([np.vstack([naver_sise_time(stock=s, date=d) for d in days]) for s in stocks.종목코드])
    
    df_list = []
    total = stocks.shape[0]
    printProgressBar(0, total, prefix = 'Progress:', suffix = 'Complete', length = 50)
    for i, s in enumerate(stocks.종목코드):
        df = naver_sise_time(stock=s, date=crdt)
        df_list.append(df)
        printProgressBar(i + 1, total, prefix = 'Progress:', suffix = 'Complete', length = 50)
    df = pd.concat(df_list, axis=1)    
    
    # Pickle로 저장
    df.to_pickle("./naver_sise_time_%s" % (crdt))

#df = pd.read_pickle("./stocks_20200525")
#df2 = pd.read_pickle("./naver_sise_time_20200519")
#df.columns
#sum(df.종목코드.value_counts()>400)
#df.체결가.plot()
#df.거래량.plot()
#df.변동량.plot()

https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=1
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=2
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=3
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=4
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=5
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=6
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=7
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=8
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=9
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=10
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=11
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=12
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=13
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=14
https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=15
http